In [3]:
import pandas as pd
import math
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

sns.set_theme(style="ticks")

pd.set_option("display.max_columns", None) 
#pd.set_option("display.max_rows", None) 

In [4]:
df_baseline_rank = pd.read_csv("../baseline_rank_trace.csv")
df_gk_rank = pd.read_csv("../gk_rank_trace.csv")
df_qdig_weight_rank = pd.read_csv("../qdig_weight_rank_trace.csv")
#df_baseline_quant = pd.read_csv("../baseline_quant_trace.csv")
#df_gk_quant = pd.read_csv("../baseline_quant_trace.csv")
#df_qdig_capacity_quant = pd.read_csv("../baseline_quant_trace.csv")
#df_qdig_update_quant = pd.read_csv("../baseline_quant_trace.csv")
#df_qdig_weight_quant = pd.read_csv("../baseline_quant_trace.csv")

In [5]:
df_gk_rank.columns

Index(['id', 'val', 'epsilon', 'univ', 'query', 'time_update', 'time_query',
       'mem_usage', 'comp_count', 'comp_mem_avg', 'value', 'weight'],
      dtype='object')

In [6]:
TARGETS = df_baseline_rank["val"].unique()
TARGETS.sort()
TARGETS

array([0, 3, 4, 5, 6, 7])

In [7]:
METRICAS = ["value", "time_query", "time_update", "mem_usage", "compress_count", "compress_saved_mem_avg"]

In [8]:
df_baseline_rank = df_baseline_rank[df_baseline_rank["val"].isin(TARGETS)]
df_baseline_rank

,id,val,epsilon,univ,query,time_update,time_query,mem_usage,comp_count,comp_mem_avg,value,weight
0,6,5,0.01,10000,9999,601737,80155,28603320,0,0,3575410,3575412
1,8,4,0.10,100000,49999,600676,78779,28617720,0,0,3576994,3577212
2,4,4,0.10,100000,16666,604233,80160,28617720,0,0,3576197,3577212
3,1,0,0.01,100000,12499,614075,9512,2114328,0,0,29808,264288
4,5,6,0.20,100000,16666,612691,114447,28177448,0,0,3452021,3522178
...,...,...,...,...,...,...,...,...,...,...,...,...
3595,0,5,0.20,1000000,166666,627760,81615,28618392,0,0,3577279,3577296
3596,6,5,0.01,1000000,499999,630381,79265,28618392,0,0,3577294,3577296
3597,1,7,0.01,100000,16666,621445,108657,27368832,0,0,3203628,3421101
3598,7,6,0.20,1000000,124999,616935,132470,28412664,0,0,3524788,3551580


In [13]:

tgt_univ = df_baseline_rank.groupby(['val', 'epsilon', 'univ', 'query']).agg({'value': ['mean'], 'weight': ['mean'], 'time_update': ['mean'], 'mem_usage': ['mean']})
tgt_univ.columns = ['value_mean', 'weight_mean', 'time_update_mean', 'mem_usage_mean']
tgt_univ = tgt_univ.reset_index()
tgt_univ['error_allowed'] = tgt_univ['epsilon'] * tgt_univ['weight_mean']
tgt_univ

tgt_univ_gk = df_gk_rank.groupby(['val', 'epsilon', 'univ', 'query']).agg({'value': ['mean'], 'time_update': ['mean'], 'mem_usage': ['mean']})
tgt_univ_gk.columns = ['value_mean', 'time_update_mean', 'mem_usage_mean']
tgt_univ_gk = tgt_univ_gk.reset_index()

tgt_univ_qdig = df_qdig_weight_rank.groupby(['val', 'epsilon', 'univ', 'value', 'query']).agg({'value': ['mean'], 'time_update': ['mean'], 'mem_usage': ['mean']})
tgt_univ_qdig.columns = ['value_mean', 'time_update_mean', 'mem_usage_mean']
tgt_univ_qdig = tgt_univ_qdig.reset_index()

tgt_univ['gk_value_mean'] = tgt_univ_gk['value_mean']
tgt_univ['gk_time_update_mean'] = tgt_univ_gk['time_update_mean']
tgt_univ['gk_mem_usage_mean'] = tgt_univ_gk['mem_usage_mean']

tgt_univ['qdig_value_mean'] = tgt_univ_qdig['value_mean']
tgt_univ['qdig_time_update_mean'] = tgt_univ_qdig['time_update_mean']
tgt_univ['qdig_mem_usage_mean'] = tgt_univ_qdig['mem_usage_mean']

tgt_univ

,val,epsilon,univ,query,value_mean,weight_mean,time_update_mean,mem_usage_mean,error_allowed,gk_value_mean,gk_time_update_mean,gk_mem_usage_mean
0,0,0.01,10000,1249,3021.0,24016.0,619667.5,192152.0,240.16,3021.0,619667.5,192152.0
1,0,0.01,10000,1666,3957.0,24016.0,619172.7,192152.0,240.16,3957.0,619172.7,192152.0
2,0,0.01,10000,2499,5916.0,24016.0,618260.8,192152.0,240.16,5916.0,618260.8,192152.0
3,0,0.01,10000,4999,12041.0,24016.0,619743.7,192152.0,240.16,12041.0,619743.7,192152.0
4,0,0.01,10000,9999,24013.0,24016.0,619917.8,192152.0,240.16,24013.0,619917.8,192152.0
...,...,...,...,...,...,...,...,...,...,...,...,...
355,7,0.20,1000000,124999,3435561.0,3531028.0,621668.1,28248248.0,706205.60,3435561.0,621668.1,28248248.0
356,7,0.20,1000000,166666,3452916.0,3531028.0,621284.2,28248248.0,706205.60,3452916.0,621284.2,28248248.0
357,7,0.20,1000000,249999,3474855.0,3531028.0,622385.2,28248248.0,706205.60,3474855.0,622385.2,28248248.0
358,7,0.20,1000000,499999,3505738.0,3531028.0,619479.0,28248248.0,706205.60,3505738.0,619479.0,28248248.0
